In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4405/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_4405/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_3_6_2,0.989121,0.787050,0.905693,0.985531,0.007588,0.126416,0.080325,0.008169,0.050734,0.087108,1.010444,0.090816,107.762439,167.487354,"Hidden Size=[12], regularizer=0.02, learning_r..."
1,model_3_5_7,0.987976,0.787045,0.904612,0.983139,0.008387,0.126419,0.081246,0.006495,0.053332,0.091578,1.011543,0.095477,107.562240,167.287155,"Hidden Size=[12], regularizer=0.02, learning_r..."
2,model_3_6_6,0.989960,0.786988,0.906479,0.986638,0.007003,0.126453,0.079655,0.007544,0.048735,0.083683,1.009639,0.087245,107.922900,167.647816,"Hidden Size=[12], regularizer=0.02, learning_r..."
3,model_3_5_3,0.986976,0.786955,0.903660,0.981793,0.009084,0.126473,0.082057,0.007014,0.055487,0.095309,1.012503,0.099367,107.402517,167.127432,"Hidden Size=[12], regularizer=0.02, learning_r..."
4,model_3_6_9,0.990546,0.786925,0.907032,0.987410,0.006594,0.126491,0.079185,0.007108,0.047287,0.081203,1.009076,0.084660,108.043197,167.768113,"Hidden Size=[12], regularizer=0.02, learning_r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,model_1_6_4,0.986591,-1.095996,0.354396,0.981536,0.009352,1.244274,0.549887,0.008297,0.045532,0.096707,0.954027,0.100824,43.344272,64.065161,"Hidden Size=[4], regularizer=0.02, learning_ra..."
696,model_1_7_1,0.988926,-1.125791,0.352966,0.965529,0.007724,1.261961,0.551106,0.016941,0.042595,0.087886,0.962031,0.091628,43.726853,64.447742,"Hidden Size=[4], regularizer=0.02, learning_ra..."
697,model_1_6_0,0.981190,-1.152007,0.388416,0.985859,0.013120,1.277524,0.520912,0.006354,0.047498,0.114541,0.935507,0.119418,42.667277,63.388166,"Hidden Size=[4], regularizer=0.02, learning_ra..."
698,model_1_5_9,0.978477,-1.156843,0.401850,0.857988,0.015012,1.280395,0.509469,0.062251,0.051310,0.122522,0.926207,0.127738,42.397870,63.118759,"Hidden Size=[4], regularizer=0.02, learning_ra..."
